In [509]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import namedtuple, deque
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Ustawienia
BATCH_SIZE = 64
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10
LEARNING_RATE = 0.001
MEMORY_SIZE = 10000
NUM_EPISODES = 500
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")




In [510]:
# Create a DataFrame 100x 4 list corStart corsStop timestart timeStop

df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=['corStart', 'corsStop', 'timestart', 'timeStop'])

#add new column sum
df['sum'] = df['timestart'] + df['timeStop']
#convert to tensor
df = torch.tensor(df.values).float()
df.shape


torch.Size([100, 5])

In [511]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [512]:
def SUMO_RUN():  # add action
    df = pd.DataFrame(
        np.random.randint(0, 100, size=(100, 4)),
        columns=["corStart", "corsStop", "timestart", "timeStop"],
    )

    df["sum"] = df["timestart"] + df["timeStop"]

    tensor = torch.tensor(df.values, dtype=torch.float32)


    return tensor

In [513]:
def sum_by_time_intervals(df, time_col_name, sum_col_name, interval):
    # Oblicz maksymalną wartość w kolumnie czasowej, aby określić, ile przedziałów potrzebujemy
    max_time = df[time_col_name].max()
    # Obliczamy liczbę przedziałów na podstawie maksymalnego czasu i długości przedziału
    num_intervals = int(np.ceil(max_time / interval))
    
    # Tworzymy wektor zer o długości liczby przedziałów do przechowywania sum
    vec_of_sum = np.zeros(num_intervals)
    
    # Iterujemy przez każdy wiersz DataFrame
    for i in range(len(df)):
                # Oblicz indeks przedziału dla aktualnego wiersza
        interval_index = int(df[time_col_name][i] // interval)
        # Upewnij się, że nie przekraczamy zakresu wektora
        if interval_index < num_intervals:
            # Dodaj wartość z kolumny sum do odpowiedniego przedziału
            vec_of_sum[interval_index] += df[sum_col_name][i]
            
    return vec_of_sum

In [514]:
!nvidia-smi


Sun Apr  7 05:09:00 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:01:00.0 Off |                  Off |
| 33%   32C    P8    16W / 260W |   1657MiB / 49152MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [515]:
actions = torch.zeros(len(state))
actions[0] = 1
actions[0]

tensor(1.)

In [ ]:
!pip install 

In [516]:
def for_models(model, state):
    add_tim = [-30, -25, -20, -15, -10, -5, 0, 5, 10, 15, 25, 20, 30]
    add_tim = torch.tensor(add_tim)
    actions = torch.zeros(len(state))
    for i in range(0, len(state)):
        value, index = model(state[i]).max(dim=0)

        x = add_tim[index]
        actions[i] = state[i][2] + add_tim[index]
    return actions

In [517]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

def SUMO(state, action):  #TODOOOOOOOOOOOOO
    # integral_t0=np.trapz(sum_by_time_intervals(state,'timeStop', 'sum', 10), dx=1)
    new_state =SUMO_RUN().to(DEVICE)
    # integral_t1=np.trapz(sum_by_time_intervals(new_state,'timeStop', 'sum', 10), dx=1)

    # if integral_t0>integral_t1:
    #     reward = 1
    # else:
    #     reward = -1
    reward =1 

    return new_state, reward

In [518]:
tensor_in = torch.zeros(5)
tensor_out = torch.zeros(12)


In [519]:
df= df.flatten()
policy_net = DQN(len(tensor_in), len(tensor_out)).to(DEVICE)
target_net = DQN(len(tensor_in), len(tensor_out)).to(DEVICE)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=LEARNING_RATE)
memory = ReplayMemory(MEMORY_SIZE)

steps_done = 0

# def select_action(state):
#     state = state.flatten()
#     global steps_done
#     sample = random.random()
#     eps_threshold = EPS_END + (EPS_START - EPS_END) * \
#         np.exp(-1. * steps_done / EPS_DECAY)
#     steps_done += 1
#     if sample > eps_threshold:
#         with torch.no_grad():
#             print(state.shape)
#             value, index = policy_net(state).max(dim=0)
#             return index

#     else:
#         return torch.tensor([[random.randrange(2)]], device=DEVICE, dtype=torch.long)


In [525]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(
        tuple(map(lambda s: s is not None, batch.next_state)),
        device=DEVICE,
        dtype=torch.bool,
    )
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])


    state_batch = torch.cat(batch.state).to(DEVICE)
    action_batch = torch.cat(batch.action).to(DEVICE)
    reward_batch = torch.cat(batch.reward).to(DEVICE)

    state_action_values = for_models(policy_net.to(DEVICE), state_batch.to(DEVICE)).to(DEVICE).gather(1, action_batch.to(DEVICE)).to(DEVICE)

    next_state_values = torch.zeros(BATCH_SIZE, device=DEVICE)
    next_state_values[non_final_mask.to(DEVICE)] = (
        for_models(target_net.to(DEVICE), non_final_next_states.to(DEVICE)).max(1)[0].detach().to(DEVICE)
    )

    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    loss = nn.SmoothL1Loss()(
        state_action_values, expected_state_action_values.unsqueeze(1)
    )

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [526]:
from itertools import count
i =0 
for i_episode in range(NUM_EPISODES):
    state = SUMO_RUN().to(DEVICE)
    print(i)
    i+=1
    for t in count():
        action = for_models(policy_net, state)
        next_state, reward = SUMO(state, action)
        

        reward = torch.tensor([reward], device=DEVICE)
        
        memory.push(state, action, next_state, reward)
        # print(state.shape, action.shape, next_state.shape, reward)
        
        state = next_state
        optimize_model()
        
        
            
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Training complete')


0
tensor([ 78., 123.,  81.,  ...,  43.,  91.,  29.], device='cuda:0')
torch.Size([6400])
6400


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)